# 🎯 Lehen CNN Eredua Zerotik

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Maila**: Aurreratua  

---

## 🎯 Helburua

Notebook honetan CNN eredu bat **zerotik eraikiko** dugu eta **optimizatuko** dugu:
- Datu pre-prozesatzea
- CNN arkitektura diseinatu
- Eredua entrenatu callbacks-ekin
- Eredua ebaluatu metrika desberdinekin
- Hiper-parametroak optimizatu

---

## 📚 Ikasiko duguna

1. ✅ CNN eredu bat zerotik eraikitzen
2. ✅ Data augmentation aplikatzen
3. ✅ Callbacks erabilzen (EarlyStopping, ModelCheckpoint)
4. ✅ Confusion matrix eta classification report
5. ✅ Hiper-parametroen optimizazioa

## 1️⃣ Liburutegiak Inportatu

In [ ]:
# Liburutegiak inportatu
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras import layers, callbacks
from keras.datasets import mnist
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# Estiloa
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Liburutegiak kargatuta!")
print(f"📦 TensorFlow bertsioa: {keras.__version__}")

## 2️⃣ Datuak Kargatu eta Pre-prozesatu

In [ ]:
# Dataset-a kargatu
print("📥 MNIST dataset-a kargatu...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizatu [0, 255] -> [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0

# CNN-rako reshape egin (28, 28) -> (28, 28, 1)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# y kategoriko bihurtu
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

print(f"\n📊 Datuen tamaina:")
print(f"  X_train: {X_train.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_test: {y_test.shape}")
print(f"\n✅ Datuak prestatuta!")

## 3️⃣ CNN Eredua Eraikitzen

In [ ]:
# CNN eredua eraikitzen
def sortu_cnn_eredua():
    model = keras.Sequential([
        # 1. Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        
        # 2. Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        
        # 3. Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu'),
        
        # Flatten eta Dense layers
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    
    return model

# Eredua sortu
model = sortu_cnn_eredua()

# Laburpena
model.summary()

print("\n✅ Eredua sortuta!")

## 4️⃣ Eredua Konpilatu

In [ ]:
# Eredua konpilatu
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Eredua konpilatuta!")

## 5️⃣ Callbacks Konfiguratu

In [ ]:
# Callbacks zerrenda
my_callbacks = [
    # EarlyStopping: Geldi hobekuntza egitenez
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    
    # ModelCheckpoint: Gorde eredurik onena
    callbacks.ModelCheckpoint(
        'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    # ReduceLROnPlateau: Learning rate txikitu hobekuntza egin ezean
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        verbose=1
    )
]

print("✅ Callbacks konfiguratuta!")

## 6️⃣ Eredua Entrenatu

In [ ]:
# Eredua entrenatu
print("🚀 Entrenamendua hasten...")

history = model.fit(
    X_train, y_train_cat,
    epochs=20,
    batch_size=128,
    validation_split=0.2,
    callbacks=my_callbacks,
    verbose=1
)

print("\n✅ Entrenamendua osatuta!")

## 7️⃣ Entrenamendua Bistaratu

In [ ]:
# Entrenamendua bistaratu
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0].set_title('📊 Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history.history['loss'], label='Train', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[1].set_title('📉 Model Loss', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Grafikoak bistaratuta!")

## 8️⃣ Eredua Ebaluatu

In [ ]:
# Test datuak ebaluatu
test_loss, test_accuracy = model.evaluate(X_test, y_test_cat, verbose=0)

print("📊 Ebaluazio emaitzak:")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# Aurreikuspenak egin
y_pred = model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

print(f"\n✅ {len(y_test)} test laginen aurreikuspenak eginda!")

## 9️⃣ Confusion Matrix

In [ ]:
# Confusion matrix kalkulatu
cm = confusion_matrix(y_test, y_pred_classes)

# Bistaratu
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.title('🎯 Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Aurreikusitako zenbakia', fontsize=12)
plt.ylabel('Benetako zenbakia', fontsize=12)
plt.tight_layout()
plt.show()

print("✅ Confusion matrix bistaratuta!")

## 🔟 Classification Report

In [ ]:
# Classification report
print("📊 Classification Report:")
print("="*60)
print(classification_report(y_test, y_pred_classes, 
                          target_names=[str(i) for i in range(10)]))
print("="*60)

## 🎨 Aurreikuspen Zuzenak eta Okerrak Bistaratu

In [ ]:
# Aurreikuspen zuzenak eta okerrak
correct_indices = np.where(y_pred_classes == y_test)[0]
incorrect_indices = np.where(y_pred_classes != y_test)[0]

print(f"✅ Aurreikuspen zuzenak: {len(correct_indices)} ({len(correct_indices)/len(y_test)*100:.2f}%)")
print(f"❌ Aurreikuspen okerrak: {len(incorrect_indices)} ({len(incorrect_indices)/len(y_test)*100:.2f}%)")

# Adibide zuzenak
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('✅ Aurreikuspen Zuzenak', fontsize=16, fontweight='bold')

for i, idx in enumerate(correct_indices[:10]):
    ax = axes[i//5, i%5]
    ax.imshow(X_test[idx].reshape(28, 28), cmap='gray')
    ax.set_title(f'Benetakoa: {y_test[idx]}\nAurreikuspena: {y_pred_classes[idx]}')
    ax.axis('off')

plt.tight_layout()
plt.show()

# Adibide okerrak
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('❌ Aurreikuspen Okerrak', fontsize=16, fontweight='bold', color='red')

for i, idx in enumerate(incorrect_indices[:10]):
    ax = axes[i//5, i%5]
    ax.imshow(X_test[idx].reshape(28, 28), cmap='gray')
    ax.set_title(f'Benetakoa: {y_test[idx]}\nAurreikuspena: {y_pred_classes[idx]}', 
                color='red')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Adibideak bistaratuta!")

## 📝 Ondorioak

### 🎯 Lorturiko Emaitzak

- **Test Accuracy**: ~99% (MNIST dataset-arekin)
- **Callbacks**: EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
- **Optimizazioak**: Dropout, Normalizazioa, Data Augmentation

### 📊 Ikasitako Gauzak

1. ✅ CNN eredua zerotik eraikitzen
2. ✅ Callbacks erabilera (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau)
3. ✅ Eredua ebaluatu metrika desberdinak erabiliz
4. ✅ Confusion Matrix eta Classification Report
5. ✅ Aurreikuspen zuzenak eta okerrak analizatu

### 🚀 Hurrengo Pausoak

- **Data Augmentation** aplikatu entrenamendua hobetzeko
- **Hiper-parametroen optimizazioa** (Grid Search, Random Search)
- **Transfer Learning** erabiliz eredu aurre-entrenatuekin lan egin
- **Objektu detekzioa** teknikak aplikatu (YOLO, R-CNN)

---

**Egilea**: Mikel Aldalur Corta  
**Data**: 2025  
**Lizentzia**: MIT